In [1]:
import pandas as pd
import os
import numpy as np

In [2]:
from byc import file_management as fm
from byc import alignment as a
from byc import database as db

from importlib import reload
reloads = [fm, a, db]
for module in reloads:
    reload(module)

## Reshaping byc timelapse data

### Exporting data from NIS elements:

Export timelapse data to tifs with order timepoint -> xy -> z. This will give output that 
should look something like:

> 20190830_byc_001t076xy08.tif

the 001 is automatically added to the filename for the experiment in NIS elements.

Sort the tifs into xy directories with shell. I should really just do this in python or a shell script but I'm moving away from NIS anyway.

```sh
mkdir 20190830_byc
mkdir 20190830_byc/tifs
# Export NIS experiment as tifs into 20190830_byc/tifs
cd 20190830_byc/tifs
mkdir 20190830_byc_xy01
mv *xy01.tif 20190830_byc_xy01
```

and so on for all xy FOVs collected. I don't usually collect multiple z stacks in my experiments, but if I do I usually just use one of them. Individual xy directories should only contain one z level

You can then move all the new stacks of tifs into a single folder called `alignment_input` so you'll have tree like:

```sh
cd 20190830_byc
tree -a
# this .nd2 is the file
# that NIS elements will output. If there are more 
# than one, export all of them into the same tifs
# folder. They individual tifs will sort properly
├── 20190830_byc_001.nd2
├── 20190830_byc_notes.md
└── tifs
    ├── alignment_input
    │   ├── 20190830_byc_xy01
    │   │   ├── 20190830_byc_001t001xy01.tif
    │   │   ├── 20190830_byc_001t002xy01.tif
    │   │   └── 20190830_byc_001t003xy01.tif
    │   └── 20190830_byc_xy02
    │       ├── 20190830_byc_001t001xy02.tif
    │       ├── 20190830_byc_001t002xy02.tif
    │       └── 20190830_byc_001t003xy02.tif
    └── alignment_output
```

Then run alignment (in byc/bin/) and follow prompts: 

```sh
(byc_env) python run_alignment.py
```

Select alignment_input and alignment_output folders, then indicate channel names in order of collection (each tif in NIS elements output is a stack of channels typically in order bf, yfp, dsred being frame 0, 1, 2 of the stack). I might just change input and output folders and channel names to  argument variables for this script.

This will read in tifs, one FOV at a time and register the stack according to features found in the bf stack. Once registered, each channels stack will be saved in `alignment_output` as `20190830_byc_xy00_bf_stack.tif`, `20190830_byc_xy00_yfp_stack.tif`, `20190830_byc_xy00_dsred_stack.tif` etc. for each channel and each xy FOV collected

### Collecting and exporting data with micromanager:

For micromanager experiments, the basic process is set collection to make a separate tif for every unique image (in order: expt_name, xy, timepoint, channel. Then, using `file_management.rename_byc()` reshape the data into the same shape as NIS elements output as indicated above. I need to take better control of `rename_byc()` handles the order of channels in the stack. I really need to get my hands on some raw micromanager output data because `rename_byc()` deletes original data as it gets renamed.

For now, when naming channels once `python run_alignment` has been called and input/output dirs chosen, write names of fluorophores in alphabetical order because that should be the order in which they're read in in `file_management.reshape_timepoints()`